<a href="https://colab.research.google.com/github/niteshnagreddy/Group3A-Capstone/blob/DhineshKumarGaneshan/PneumoniaDetectionModelEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os

working_dir = '/content/drive/My Drive/PneumoniaDetection'
os.chdir(working_dir)

In [0]:
import zipfile

In [0]:
zip_file_dir = working_dir+'/trainData.zip'

In [0]:
#extracted = zipfile.ZipFile(zip_file_dir, 'r')
#extracted.extractall(path=working_dir)

In [0]:
import numpy as np
data = np.load("./trainDataSample.npy", allow_pickle=True)

In [0]:
type(data)

In [0]:
data.shape

In [0]:
data.view()[0]

In [0]:
data[0][1]

In [0]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

In [0]:
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input

In [0]:
masks = np.zeros((int(data.shape[0]), IMAGE_HEIGHT, IMAGE_WIDTH))
X_train = np.zeros((int(data.shape[0]), IMAGE_HEIGHT, IMAGE_WIDTH, 3))
for index in range(data.shape[0]):
    img = data[index][0]
    img = cv2.resize(img, dsize=(IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=cv2.INTER_CUBIC)
    try:
      img = img[:, :, :3]
    except:
      continue
    X_train[index] = preprocess_input(np.array(img, dtype=np.float32))
    for i in data[index][1]:
        x1 = int(i["points"][0]['x'] * IMAGE_WIDTH)
        x2 = int(i["points"][1]['x'] * IMAGE_WIDTH)
        y1 = int(i["points"][0]['y'] * IMAGE_HEIGHT)
        y2 = int(i["points"][1]['y'] * IMAGE_HEIGHT)
        masks[index][y1:y2, x1:x2] = 1

In [0]:
X_train.shape

In [0]:
type(masks)

In [0]:
masks.shape

In [0]:
masks.shape[0]

In [0]:
from matplotlib import pyplot
n = 10
print(X_train[n])
pyplot.imshow(X_train[n])

In [0]:
pyplot.imshow(masks[n])

In [0]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.layers import Concatenate, UpSampling2D, Conv2D, Reshape, concatenate
from tensorflow.keras.models import Model
import tensorflow as tf
import cv2

In [0]:
model = MobileNet()

In [0]:
model.summary()

In [0]:
from keras import backend as K
K.set_image_data_format('channels_first')

In [0]:
def create_model(trainable=True):
    # model = #### Add your code here ####
    model = MobileNet(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3), 
                      include_top=False, alpha=1.0, weights='imagenet')
    for layer in model.layers:
        layer.trainable = trainable

    # Add all the UNET layers here
    #### Add your code here ####

    # getting the layers from mobilenet network
    conv_pw_13_relu = model.get_layer("conv_pw_13_relu").output
    conv_pw_12_relu = model.get_layer("conv_pw_12_relu").output
    conv_pw_11_relu = model.get_layer("conv_pw_11_relu").output
    conv_pw_10_relu = model.get_layer("conv_pw_10_relu").output
    conv_pw_9_relu = model.get_layer("conv_pw_9_relu").output
    conv_pw_8_relu = model.get_layer("conv_pw_8_relu").output
    conv_pw_7_relu = model.get_layer("conv_pw_7_relu").output
    conv_pw_6_relu = model.get_layer("conv_pw_6_relu").output
    conv_pw_5_relu = model.get_layer("conv_pw_5_relu").output
    conv_pw_4_relu = model.get_layer("conv_pw_4_relu").output
    conv_pw_3_relu = model.get_layer("conv_pw_3_relu").output
    conv_pw_2_relu = model.get_layer("conv_pw_2_relu").output
    conv_pw_1_relu = model.get_layer("conv_pw_1_relu").output
    input_1 = model.layers[0].output

    
    # Adding Unet layers
    # Each set will have 1 upsampling, then concat with the mobilenet layers having same shape
    # followed by 2 conved layers with extra parameters

    up2 = UpSampling2D()(conv_pw_13_relu)
    concat1 = Concatenate()([up2, conv_pw_11_relu])
    new_conv15 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat1)
    new_conv15 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(new_conv15)

    up3 = UpSampling2D()(concat1)
    concat2 = Concatenate()([up3, conv_pw_5_relu])
    new_conv16 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat2)
    new_conv16 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(new_conv16)

    up4 = UpSampling2D()(concat2)
    concat3 = Concatenate()([up4, conv_pw_3_relu])
    new_conv17 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat3)
    new_conv17 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(new_conv17)

    up5 = UpSampling2D()(concat3)
    concat4 = Concatenate()([up5, conv_pw_1_relu])
    new_conv17 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat4)
    new_conv17 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(new_conv17)

    up6 = UpSampling2D()(concat4)
    concat5 = Concatenate()([up6, input_1])

    outputs = Conv2D(1, kernel_size=1, activation="sigmoid")(concat5)
    outputs = Reshape((IMAGE_HEIGHT, IMAGE_WIDTH))(outputs)

    # #### Add your code here ####
    return Model(inputs=model.input, outputs=outputs)

In [0]:
model = create_model(True)

In [0]:
model.summary()

In [0]:
from keras import backend as K

def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    n = K.sum(y_true_f * y_pred_f)
    d = K.sum(y_true_f) + K.sum(y_pred_f)
    return n / (d + K.epsilon())

In [0]:
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.backend import log, epsilon
def loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - log(dice_coefficient(y_true, y_pred) + epsilon())

In [0]:
from tensorflow.keras.optimizers import Adam
# optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss=loss, optimizer="adam", metrics=[dice_coefficient])

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint("model-{loss:.2f}.h5", monitor="loss", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="min", save_freq=1)
stop = EarlyStopping(monitor="loss", patience=5, mode="min")
reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.2, patience=5, min_lr=1e-7, verbose=1, mode="min")

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [0]:
EPOCHS = 50
BATCH_SIZE = 8
callbacks = [checkpoint, reduce_lr, stop]


# model.fit(X_tr,y_tr, validation_data=(X_v,y_v), epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle='batch', callbacks=callbacks, verbose=1)

model.fit(X_train, masks, validation_split=0.3, 
          epochs=EPOCHS, batch_size=BATCH_SIZE, 
          shuffle='batch', 
          callbacks=callbacks, 
          verbose=1)

In [0]:
WEIGHTS_FILE = "model-16.15.h5"

In [0]:
model = create_model(True)
model.load_weights(WEIGHTS_FILE)

In [0]:
n = 35
sample_image = X_train[n]

In [0]:
pyplot.imshow(sample_image)

In [0]:
image_height, image_width, _ = sample_image.shape
img = cv2.resize(sample_image, (IMAGE_HEIGHT, IMAGE_HEIGHT), 3)
scaled = preprocess_input(np.array(img, dtype=np.float32))
# scaled_exp = np.expand_dims(scaled, axis=1)
# scaled_v = np.vectorize(scaled)

bbox_pred = model.predict(x=np.array([img]))[0]



x0 = ((bbox_pred[0]) * image_width / IMAGE_HEIGHT)
# x0 = x0.astype(int)
y0 = ((bbox_pred[1]) * image_height / IMAGE_HEIGHT)
# y0 = y0.astype(int)
x1 = ((bbox_pred[2]) * image_width / IMAGE_HEIGHT)
# x1 = x1.astype(int)
y1 = ((bbox_pred[3]) * image_height / IMAGE_HEIGHT)
# y1 = y1.astype(int)

In [0]:
pyplot.imshow(bbox_pred)

In [0]:
unscaled = X_train[n]

image = cv2.resize(unscaled, (IMAGE_WIDTH, IMAGE_HEIGHT),3)

feat_scaled = preprocess_input(np.array(unscaled, dtype=np.float32))

pred_mask = cv2.resize(1.0*(model.predict(x=np.array([image]))[0] > 0.5), (IMAGE_WIDTH,IMAGE_HEIGHT))

image2 = image
image2[:,:,0] = pred_mask*image[:,:,0]
image2[:,:,1] = pred_mask*image[:,:,1]
image2[:,:,2] = pred_mask*image[:,:,2]

out_image = image2

pyplot.imshow(out_image)
# cv2.imshow("Predicted Mask", out_image)
cv2.waitKey(0)
cv2.destroyAllWindows()